# Импорты

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Курсовая

/content/drive/MyDrive/Курсовая


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import re
import random, os
import pickle

from tqdm.notebook import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
import gensim.downloader as api
from transformers import BertModel, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

# Подготовка

In [ ]:
df = pd.read_csv('sentiment140_100000.csv')

In [ ]:
knowledge_dct = {}

In [ ]:
# берем только обучающую выборку для получения эмбеддингов
train = df[df["data_type"]=='train'].reset_index(drop=True)

In [ ]:
# подсчитаем количество слов в каждом твите
count_vectorizer = CountVectorizer(min_df=10, max_df=0.99)
count_data = count_vectorizer.fit_transform(train['tweet'].values)

# Создание эмбеддингов для юзеров с помощью LDA

In [ ]:
# расмотрим количество тем 2, 5, 10, 20
n_components_list = [2, 5, 10, 20]

## Post-LDA

То есть считаем, что каждый твит - это отдельный документ

Смотрим на распределение тем в каждом документе

Эмбеддинг юзера - среднее всех распределений его документов

In [ ]:
for n_components in n_components_list:
  lda = LatentDirichletAllocation(n_components=n_components, random_state=42, n_jobs=-1)
  transformed_train = lda.fit_transform(count_data)

  knowledge_dct[f'post_lda_{n_components}'] = {}

  for user, group in train.groupby(by=['user']):
    embeddings_for_user = transformed_train[group.index]
    knowledge_dct[f'post_lda_{n_components}'][user[0]] = embeddings_for_user.mean(axis=0)

## User-LDA

То есть считаем, что все твиты одного пользователя - это один документ

Эмбеддинг юзера - распределение тем в этом одном документе

In [ ]:
# новая матрица
new_count_data = []

In [ ]:
# в каждой строке теперь по сути объединенные твиты от каждого пользователя
for user, group in train.groupby(by=['user']):
  new_count_data.append(count_data[group.index].sum(axis=0).tolist()[0])

In [ ]:
new_count_data = np.array(new_count_data)

In [ ]:
len(new_count_data), df['user'].nunique()

(1501, 1501)

In [ ]:
for n_components in n_components_list:
  lda = LatentDirichletAllocation(n_components=n_components, random_state=42, n_jobs=-1)
  transformed_train = lda.fit_transform(new_count_data)

  knowledge_dct[f'user_lda_{n_components}'] = {}

  for i, (user, group) in enumerate(train.groupby(by=['user'])):
    embedding_for_user = transformed_train[i]
    knowledge_dct[f'user_lda_{n_components}'][user[0]] = embedding_for_user

# Создание эмбеддингов для юзеров с помощью Word2Vec и GloVe

In [ ]:
# расcмотрим предобученные модели из библиотеки gensim с размерностями эмбеддингов 100 и 200: glove-twitter-100 и glove-twitter-200
# а также сами обучим модель с эмбеддингами размерности 300 на данных

model_list = [
  api.load("glove-twitter-100"),
  api.load("glove-twitter-200"),
  Word2Vec(
      train['tweet'].apply(lambda x: x.split(' ')).values,
      min_count=10,
      vector_size=300,
      window=5
      )
]

[==================================================] 100.0% 387.1/387.1MB downloaded
[==================================================] 100.0% 758.5/758.5MB downloaded


In [ ]:
# слова из словаря
features_names = count_vectorizer.get_feature_names_out()

In [ ]:
# считаем эмбеддинги следующим образом
# 1. для каждого пользователя выбираем все его твиты
# 2. выбираем все порядковые номера слов, которые использует пользователь (учитывая повторения)
# 3. с помощью порядковых номеров из словаря выбираем слова (как строки)
# 4. считаем эмбеддинги по модели
for model, name in zip(model_list, ['glove_100', 'glove_200', 'w2v_300']):
  knowledge_dct[name] = {}

  for user in train['user'].unique():
    user_tweet_idxes = train[train["user"]==user].index
    user_count_data = count_data[user_tweet_idxes].toarray()
    user_words_idxes = np.where(user_count_data >= 1)[1]
    user_words = features_names[user_words_idxes]

    if name == 'w2v_300':
      user_embedding = model.wv[user_words].mean(axis=0)

    else:
      user_embedding = []

      # так как модель обучена не на нашем корпусе, будет включать только те слова
      # которые там есть
      for user_word in user_words:
        if user_word in model:
          user_embedding.append(model[user_word])

      user_embedding = np.array(user_embedding)
      user_embedding = user_embedding.mean(axis=0)

    knowledge_dct[name][user] = user_embedding

# Создание эмбеддингов для юзеров с помощью BERT

По сути расчет аналогичен Word2Vec и GloVe, только в данном случае вектор будет фиксированного размера - 768.

In [ ]:
with open('knowledge.pickle', 'rb') as f:
  knowledge_dct = pickle.load(f)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device);

In [ ]:
max_length = max(train['tweet'].apply(lambda x: len(x.split(' '))).value_counts().index)

In [ ]:
knowledge_dct['bert'] = {}

In [ ]:
for user, group in tqdm(train.groupby(by=['user'])):
  encoder = tokenizer.batch_encode_plus(
      group['tweet'].values,
      add_special_tokens = True,
      return_attention_mask = True,
      truncation = True,
      padding = 'max_length',
      max_length = max_length,
      return_tensors = 'pt'
      )

  dataset = TensorDataset(encoder['input_ids'], encoder['attention_mask'])
  dataloader = DataLoader(
    dataset,
    sampler=SequentialSampler(dataset),
    batch_size=32
  )

  lst = []

  for batch in dataloader:
    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids': batch[0],
        'attention_mask': batch[1],
        }

    with torch.no_grad():
      last_hidden_states = model(**inputs)[0]

    lst.append(last_hidden_states)

  user_embedding = torch.concat(lst).mean(axis=(0,1))
  knowledge_dct['bert'][user[0]] = np.array(user_embedding.cpu())

  0%|          | 0/1501 [00:00<?, ?it/s]

In [ ]:
# сохраним словарь
with open('knowledge.pickle', 'wb') as f:
  pickle.dump(knowledge_dct, f)